In [1]:
# Attempting to build a network that would work
# for all the cases of the damped harmonic oscillator

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivps

ImportError: cannot import name 'solve_ivps' from 'scipy.integrate' (/Users/chamundeshwarivadamalai/Desktop/NeuralNetworks_DiffEqn/nn_diff/lib/python3.13/site-packages/scipy/integrate/__init__.py)